In [16]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/combined_data.csv', index_col=0) # , nrows=1000)
data = data[data['contest_type'].isin(['Classic', 'Showdown'])].copy()
# data = data[data['contest_id'] == data['contest_id'][0]]
display(data)

,contest_id,roster_slot_id,player_id,entry_count,usage_ratio,season,double_up,draft_group_id,fifty_fifty,guaranteed,...,id,team_id,team_abbreviation,competition_id,player_name,position,salary,swappable,disabled,projection
0,163439930,QB,403308,934,0.011777,2024,f,109136,f,t,...,35213654,323,ATL,6018721,Kirk Cousins,QB,6100.0,t,f,16.66
1,163439930,QB,496083,934,0.002141,2024,f,109136,f,t,...,35213665,350,NO,6018919,Derek Carr,QB,5300.0,t,f,14.80
2,163439930,QB,513856,934,0.036403,2024,f,109136,f,t,...,35213660,361,SEA,6019075,Geno Smith,QB,5500.0,t,f,17.43
3,163439930,QB,591816,934,0.004283,2024,f,109136,f,t,...,35213648,331,DAL,6018551,Dak Prescott,QB,7100.0,t,f,17.03
4,163439930,QB,607047,934,0.001071,2024,f,109136,f,t,...,35213671,348,NE,6018799,Jacoby Brissett,QB,5100.0,t,f,13.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3039361,172306485,DST,332,99,0.151515,2024,f,119182,f,t,...,37128077,332,DEN,6020071,Broncos,DST,2800.0,t,f,5.60
3039362,172306485,DST,343,99,0.181818,2024,f,119182,f,t,...,37128075,343,LAR,6020083,Rams,DST,3200.0,t,f,6.30
3039363,172306485,DST,348,99,0.262626,2024,f,119182,f,t,...,37128079,348,NE,6020095,Patriots,DST,2400.0,t,f,5.70
3039364,172306485,DST,355,99,0.050505,2024,f,119182,f,t,...,37128078,355,ARI,6020083,Cardinals,DST,2600.0,t,f,5.00


In [ ]:
# check the data
print(data[data['salary'] == 0.0])

Empty DataFrame
Columns: [contest_id, roster_slot_id, player_id, entry_count, usage_ratio, season, double_up, draft_group_id, fifty_fifty, guaranteed, head_to_head, contest_name, payout, starred, starts_at, week, entries_max, entries_fee, contest_type, games_count, multientry, max_entry_fee, rake, id, team_id, team_abbreviation, competition_id, player_name, position, salary, swappable, disabled, projection]
Index: []

[0 rows x 33 columns]


: 

## Add some features

In [ ]:
# use this to test the computation
if data_bkp is None:
    data_bkp = data.copy()
# data = data.sample(n=1000, random_state=42).copy()

def compute_groupby_features(df):
    df['salary_mean'] = df['salary'].mean()
    df['salary_max'] = df['salary'].max()
    df['salary_std'] = df['salary'].std()
    df['projection_mean'] = df['projection'].mean()
    df['projection_max'] = df['projection'].max()
    df['projection_std'] = df['projection'].std()
    df['projection_value_ratio_mean'] = df['projection_value_ratio'].mean()
    df['projection_value_ratio_max'] = df['projection_value_ratio'].max()
    df['projection_value_ratio_std'] = df['projection_value_ratio'].std()

    return df

# player_groups = data[['contest_id', 'roster_slot_id', 'player_id', 'salary', 'projection']].copy()
data['projection_value_ratio'] = data['projection'] / data['salary'] * 50000
player_groups = data.groupby(['contest_id', 'roster_slot_id']).apply(compute_groupby_features, include_groups=False).reset_index()

player_groups['salary_vor'] = (player_groups['salary'] - player_groups['salary_mean']) / player_groups['salary_std']
player_groups['projection_vor'] = (player_groups['projection'] - player_groups['projection_mean']) / player_groups['projection_std']
player_groups['projection_value_ratio_vor'] = (player_groups['projection_value_ratio'] - player_groups['projection_value_ratio_mean']) / player_groups['projection_value_ratio_std']

player_groups['salary_vum'] = (player_groups['salary'] - player_groups['salary_max']) / player_groups['salary_std']
player_groups['projection_vum'] = (player_groups['projection'] - player_groups['projection_max']) / player_groups['projection_std']
player_groups['projection_value_ratio_vum'] = (player_groups['projection_value_ratio'] - player_groups['projection_value_ratio_max']) / player_groups['projection_value_ratio_std']

display(player_groups)
player_groups.to_csv('data/project_data.csv')

In [ ]:
display(player_groups.reset_index())

,index,contest_id,roster_slot_id,level_2,player_id,entry_count,usage_ratio,season,double_up,draft_group_id,...,projection_std,projection_value_ratio_mean,projection_value_ratio_max,projection_value_ratio_std,salary_vor,projection_vor,projection_value_ratio_vor,salary_vum,projection_vum,projection_value_ratio_vum
0,0,163439930,DST,256,362,934,0.051392,2024,f,109136,...,1.049974,107.834930,126.666667,11.036855,0.995198,0.630968,-0.451015,-0.868537,-0.761924,-2.157274
1,1,163439930,DST,237,324,934,0.084582,2024,f,109136,...,1.049974,107.834930,126.666667,11.036855,-0.090473,1.011930,1.706259,-1.954208,-0.380962,0.000000
2,2,163439930,DST,247,345,934,0.007495,2024,f,109136,...,1.049974,107.834930,126.666667,11.036855,0.343796,-0.607158,-1.417744,-1.519939,-2.000049,-3.124003
3,3,163439930,DST,244,336,934,0.057816,2024,f,109136,...,1.049974,107.834930,126.666667,11.036855,-0.524741,-0.988120,-0.871683,-2.388476,-2.381011,-2.577943
4,4,163439930,DST,254,357,934,0.109208,2024,f,109136,...,1.049974,107.834930,126.666667,11.036855,-0.307607,0.726208,1.633358,-2.171342,-0.666683,-0.072901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,164120406,WR,937,1178741,14858,0.153453,2024,f,109136,...,4.714444,86.406672,159.166667,31.034075,0.880563,1.350984,1.222097,-1.926098,-1.389347,-1.122422
996,996,164120406,WR,897,884943,14858,0.004375,2024,f,109136,...,4.714444,86.406672,159.166667,31.034075,-0.514198,-0.263204,0.235589,-3.320859,-3.003535,-2.108931
997,997,164120406,WR,875,742382,14858,0.000135,2024,f,109136,...,4.714444,86.406672,159.166667,31.034075,-0.647032,-1.236808,-1.599852,-3.453693,-3.977139,-3.944371
998,998,164120406,WR,955,1322431,14858,0.011172,2024,f,109136,...,4.714444,86.406672,159.166667,31.034075,0.216391,0.231022,0.322011,-2.590270,-2.509310,-2.022508
